# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [244]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/Advertising.csv')
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [245]:
data.isnull().sum() # нет пропусков

Unnamed: 0    0
TV            0
radio         0
newspaper     0
sales         0
dtype: int64

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [246]:
X = np.asarray(data[['TV', 'radio', 'newspaper']])
y = np.asarray(data['sales'])


## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [247]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [248]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$ 
        
        * Вычисляем новое значение параметра $k$-ого коэффициента: 
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}w_j$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



***



Пусть у нас есть функция потерь $\large{L(w_0, w_1) = \frac{1}{2m} \sum_{i=1}^{m}(y_i - h_i)^2}$, где $h_i = w_0 + w_1 x_i$ и мы минимизируем ее по отдельным параметрам:

1. **Обновление $ \mathbf{w_0}$** при фиксированном $w_1$:

    Вычисляем производную $L$ по $w_0$ и приравниваем её к нулю:

    $$\frac{\partial L}{\partial w_0} = \frac{1}{m} \sum_{i=1}^{m} (y_i - h_i) \cdot \frac{\partial}{\partial w_0} (y_i - h_i) = 0.$$

    $$\frac{1}{m} \sum_{i=1}^{m} (h_i - y_i) = 0. \quad \left(\text{так как}\quad \frac{\partial}{\partial w_0} (y_i - h_i)\right) = -1$$ 


    $$\frac{1}{m} \sum_{i=1}^{m} (w_0 + w_1 x_i - y_i) = 0.$$

    Можем избавиться от $\frac{1}{m}$, собственно известный факт, что константы не влияют на нахождение экстремума (поэтому неважно используем мы half-MSE как я или же стандартный вариант)
    $$\sum_{i=1}^{m} (w_0 + w_1 x_i - y_i) = 0.$$

    Выражаем отсюда $w_0$:

    $$w_0 = \frac{1}{m} \sum_{i=1}^{m} (y_i - w_1 x_i).$$

2. **Обновление $ \mathbf{w_1}$** при фиксированном $w_0$
    $$\frac{\partial L}{\partial w_1} = \frac{1}{m} \sum_{i=1}^{m} (y_i - h_i) \cdot \frac{\partial}{\partial w_1} (y_i - h_i) = 0.$$

    Подставляя выражение для производной, получаем:

    $$\frac{1}{m} \sum_{i=1}^{m} (h_i - y_i) \cdot x_i = 0. \quad \left(\frac{\partial}{\partial w_1} (y_i - h_i) = -x_i\right)$$

    Подставляя $h_i$, получаем:

    $$\sum_{i=1}^{m} (w_0 + w_1 x_i - y_i) \cdot x_i = 0.$$

    Разрешая относительно $w_1$, получаем:

    $$w_1 = \frac{\sum_{i=1}^{m} x_i(y_i - w_0)}{\sum_{i=1}^{m} (x_i)^2}.$$

Тем самым, общая формула для произвольного k-того признака (при учете что в матрице $X$ присутствует столбец-константа) будет такая:
 $$w_k = \frac{\sum_{i=1}^{m} x_{ik}(y_i - h_i)}{\sum_{i=1}^{m} (x_{ik})^2} \quad h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j \\ \text{или в матричном виде} \\ w_k = \frac{x_{k}^T (y - h)}{x_k^T x_k} $$

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [249]:
num_iters = 1000
m = X.shape[0]
n = X.shape[1]
costs = []
w = np.zeros(n).reshape(-1,1)
for _ in range(num_iters):
    for k in range(n):
        h = (X[:,0:k]@ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        w[k] = (X[:,k].T @ (y-h))/(X[:,k].T @ X[:,k])
        cost = (1/(2*m)) * np.sum((X @ w - y)**2)
        costs.append(cost)
print(w)
print(f"MSE: {2*min(costs):.2f}")

[[ 41.56217205]
 [110.13144155]
 [ 73.52860638]
 [ -0.55006384]]
MSE: 2.78


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [250]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [251]:
X = np.asarray(data[['TV', 'radio', 'newspaper']])
X = (X - X.mean(axis=0))/X.std(axis=0)

**Добавим единичный столбец**

In [252]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
X.shape

(200, 4)

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [253]:
def mse_error(y_true, y_pred):
    return (1/len(y_true)) * np.sum((y_true.ravel() - y_pred.ravel())**2)

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [254]:
print(mse_error(y, y.mean()*np.ones(len(y)).reshape(-1, 1)))

27.085743750000002


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [255]:
def lin_pred(X,w):
    return X @ w

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [256]:
def stoch_grad_step(X, y, w, train_ind, eta):
    w_next = w - 2*eta * (X[train_ind,:].T * (X[train_ind,:] @ w  - y[train_ind])).reshape(-1,1)
    return w_next

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [257]:
def stochastic_gradient_descent(X, y, w_0, eta=0.01, n_iter=1000, tol=0.001, seed=42):
    np.random.seed(seed)
    w_cur = w_0
    loss_values = []
    weight_delta = np.inf
    i = 0
    while weight_delta > tol or i < n_iter:
        train_ind = np.random.randint(0, X.shape[0])
        loss_values.append(mse_error(y, lin_pred(X,w)))
        w_new = stoch_grad_step(X, y, w_cur, train_ind, eta=eta)
        weight_delta = np.linalg.norm(w_new - w_cur)
        w_cur = w_new
        i+=1
    if weight_delta < tol:
        print("Tol was reached")
    if i == n_iter:
        print("Iterations bound reached")
    return w_cur, loss_values


 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [267]:
w_0 = np.zeros((4,1))
w, errors = stochastic_gradient_descent(X, y, w_0, eta=0.0001, tol=0.00001, n_iter=10**5, seed=42)

Tol was reached


**Выведите вектор весов, к которому сошелся метод.**

In [268]:
print(w)

[[14.0058364 ]
 [ 3.95256843]
 [ 2.76774544]
 [-0.04799935]]


**Выведите среднеквадратичную ошибку на последней итерации.**

In [269]:
print(errors[-1])

2.7870070188656118


In [261]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(X,y.ravel())
print(mse_error(y, sgd.predict(X)))

2.7842894171072454


#### В целом, как с помощью координатного спуска, так и с помощью стохастического градиентного спуска собственной реализации удалось получить оптимальный вектор весов для наших данных и достичь хорошего результата по метрике MSE, почти идентичного готовым решениям, полученным с помощью библиотеки sklearn.